In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import shutil

import descarteslabs as dl
from descarteslabs.catalog import Image, properties
import geopandas as gpd
import rasterio as rs
from rasterio.merge import merge
from tqdm import tqdm

plastics_path = '/Users/ckruse/Documents/earthrise/plastics'
if plastics_path in sys.path:
    sys.path.remove('/Users/ckruse/Documents/earthrise/plastics')
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from scripts import deploy_nn_v1
from scripts import dl_utils

In [177]:
# User inputs

model_version = '1.0.0'
model_name = '25px_v1.0_2021-06-10'
model_file = '../models/' + model_name + '.h5'

roi = 'bolivar'
roi_file = f'../data/boundaries/{roi}.geojson'

# Note on dates: The date range should be longer than the spectrogram length.
# Starting on successive mosaic periods (typically: monthly), as many
# spectrograms are created as fit in the date range.
start_date = '2020-01-01'
end_date = '2021-01-01'

mosaic_period = 6
mosaic_method = 'min'
spectrogram_interval = 2

product_id = f'earthrise:{roi}_v{model_version}_{start_date}_{end_date}_period_{mosaic_period}_interval_{spectrogram_interval}' 
product_name = product_id.split(':')[-1]  # Arbitrary string - optionally set this to something more human readable.

run_local = False # If False, the model prediction tasks are async queued and sent to DL for processing.

In [178]:
args = [
    '--roi_file',
    roi_file,
    '--product_id',
    product_id,
    '--product_name',
    product_name, 
    '--model_file',
    model_file,
    '--model_name',
    model_name,
    '--mosaic_period',
    str(mosaic_period),
    '--mosaic_method',
    mosaic_method,
    '--spectrogram_interval',
    str(spectrogram_interval),
    '--start_date',
    start_date,
    '--end_date',
    end_date,
    '--tilesize',
    str(34 * 25)
]
if run_local:
    args.append('--run_local')

Launch Descartes job. Monitor at https://monitor.descarteslabs.com/

In [179]:
# Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
%cd ../scripts
%pwd

/Users/ckruse/Documents/earthrise/mining-detector/scripts


'/Users/ckruse/Documents/earthrise/mining-detector/scripts'

In [180]:
deploy_nn_v1.main(args)

Split ROI into 3579 tiles
Model 25px_v1.0_2021-06-10 found in DLStorage.
Product earthrise:bolivar_v1.0.0_2020-01-01_2021-01-01_period_6_interval_2 already exists...


100%|██████████| 3579/3579 [08:46<00:00,  6.79it/s]


Download outputs after task is complete

In [182]:
fc_id = [fc.id for fc in dl.vectors.FeatureCollection.list() if product_id in fc.id][0]
fc = dl.vectors.FeatureCollection(fc_id)
region = gpd.read_file(roi_file)['geometry'][0]
    
features = []
for elem in tqdm(fc.filter(region).features()):
    features.append(elem.geojson)
results = gpd.GeoDataFrame.from_features(features)
results.to_file(f"../data/outputs/{product_id.split(':')[-1]}.geojson", driver='GeoJSON')
print(len(features), 'features found')

100%|██████████| 76806/76806 [01:37<00:00, 790.72it/s] 


76806 features found
